In [272]:
from keras.layers import Input, Embedding, SimpleRNN, Dense, merge, Flatten, BatchNormalization, LSTM, TimeDistributed, Dropout
from keras.models import Model
from keras.optimizers import Adam
import keras
import urllib2
import numpy as np
import re
import itertools

# dataset_raw = urllib2.urlopen("https://s3.amazonaws.com/text-datasets/nietzsche.txt").read().\
#     replace('\n', ' ').lower()
with open('/home/ubuntu/trump.txt') as f:
    txt = f.read()
    txt = txt.replace('\n', ' ')
    txt = re.sub(' +', ' ', txt)
    txt = re.split('SPEECH \d ', txt)
    txt = [i.lower() for i in txt if i != '']

In [273]:
txt[0][:100]

"...thank you so much. that's so nice. isn't he a great guy. he doesn't get a fair press; he doesn't "

In [274]:
vocab = sorted(list(set([i for i in j for j in txt])))

In [253]:
vocab.insert(0, '\0')

In [276]:
txt_encoder = {v:k for k,v in enumerate(vocab)}
txt_decoder = {k:v for k,v in enumerate(vocab)}

In [280]:
datasets_encoded = [[txt_encoder[i] for i in j] for j in txt]

In [311]:
len(datasets_encoded)

9

In [285]:
dataset_encoded = datasets_encoded[0]

In [286]:
vocab_size = len(vocab)

In [287]:
seq_len = 40

In [398]:
n_fac = 60
max_len = 40

sentences = []
next_chars = []

for i in range(len(datasets_encoded)):
    sentences.append([])
    next_chars.append([])
    data = datasets_encoded[i]
    for j in range(len(data) - max_len + 1):
        sentences[i].append(data[j:j+max_len])
        next_chars[i].append(data[j+1:j+max_len+1])         
        
    sentences[i] = np.stack([np.array(o) for o in sentences[i][:-1]], axis=0)
    next_chars[i] = np.stack([np.array(o) for o in next_chars[i][:-1]], axis=0)
    
print('nb sequences: %s' % len(sentences[0]))

nb sequences: 20807


In [401]:
sentences[5].shape, next_chars[5].shape

((33279, 40), (33279, 40))

In [402]:
sentences[0].shape, next_chars[0].shape

((20807, 40), (20807, 40))

In [413]:
inp = Input(shape=(max_len,))
emb = Embedding(vocab_size, n_fac, input_length=max_len)(inp)
lstm1 = LSTM(512, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu')(emb)
d1 = Dropout(0.2)(lstm1)
lstm2 = LSTM(512, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu')(d1)
d2 = Dropout(0.2)(lstm2)
out = TimeDistributed(Dense(vocab_size, activation='softmax'))(d2)

In [414]:
mdl = Model(input=inp, output=out)

In [405]:
mdl.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_8 (InputLayer)             (None, 40)            0                                            
____________________________________________________________________________________________________
embedding_8 (Embedding)          (None, 40, 60)        4080        input_8[0][0]                    
____________________________________________________________________________________________________
lstm_14 (LSTM)                   (None, 40, 512)       1173504     embedding_8[0][0]                
____________________________________________________________________________________________________
dropout_13 (Dropout)             (None, 40, 512)       0           lstm_14[0][0]                    
___________________________________________________________________________________________

In [294]:
vocab_size

68

In [193]:
def generate_text(num_chars):
    base_str = 'suffering is not the only way to see a t'
    for i in range(num_chars):
        prediction = mdl.predict(np.array([txt_encoder[i] for i in base_str[-40:]])\
                                 [np.newaxis,:])[0][-1]
        prediction = prediction / np.sum(prediction)
        next_char = np.random.choice(vocab, p=prediction) # the final model output
        base_str = base_str + next_char
    return base_str

In [174]:
vocab_size

59

In [415]:
def print_example():
    seed_string = 'need to secure our borders and make sure'
    for i in range(320):
        x=np.array([txt_encoder[c] for c in seed_string[-40:]])[np.newaxis,:]
        preds = mdl.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = np.random.choice(vocab, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [416]:
mdl.compile(optimizer=Adam(lr=0.01), loss='sparse_categorical_crossentropy')

In [417]:
for i in range(len(sentences)):
    mdl.fit(sentences[i], np.expand_dims(next_chars[i], -1), nb_epoch=1, batch_size=64)
    print_example()

Epoch 1/1
20807/20807 [==============================] - 38s - loss: 2.5992    
need to secure our borders and make sure thing thoopen is wrreidiil. wat the have terurk. they' aversut on yor's arasks.. moan cibes liat so chaterronnciet, and hroon to mors por o2 at don't waly have a loal i we he ko te have all putore afe un mel. anl i't goallak are sore, is i ef are yout. secy, he. are bele for to; and we amwiorsou. s can is brrion and a
Epoch 1/1
26856/26856 [==============================] - 49s - loss: 1.7566    
need to secure our borders and make surents, we tone defending the world, we’re to centuresmill long and furpot and septe, and we had foreign policy prember any is their ralies, 1nd has gavely, presis propole, the world, become. ever as islad the necurity youl warther peaor lame as completed ess recourtr. ruve. we have serrorism will must these xay has als
Epoch 1/1
2693/2693 [==============================] - 5s - loss: 1.5057     
need to secure our borders and make sure t

In [409]:
print_example()

...thank you so much. that's so nice. ise e. j. we whi d csedmgdr. he sallsie tldk5t an in wilant o  meelda falhallid’tineke berned go i blangoe anthde i ffallsot, mua. se gi t meyo le n,d, w our..sg wwesnlledelll.theret. whong ussndet. aatics nu g seed ’ve dnnhnh ecyiandon, the bpe wo amo.. thmy . beve d owpanto thehpt te yieotrie. dit, thato sof de sor


In [309]:
mdl.fit(sentences, np.expand_dims(next_chars, -1), nb_epoch=1, batch_size=64)
print_example()

Epoch 1/1
20806/20806 [==============================] - 38s - loss: 1.8849    
...thank you so much. that's so nice. is i konnands to say nople about everybods can, beed it, i way. in, leads. debemy about soment dolts to do awhsed, i clople abous tile of that infhiching0 but they lost. we and fay abony that we havel, to? what's thouds; we owhat and to said have because nesords. we moln to say and it's, wered colltar have theme thouble 


In [310]:
generate_text(1000)

"suffering is not the only way to see a the money for alomportsfores, they i llalss a luch about we ford mtans, i sioming, and i buy every. and mount but ocerrender. what'n so it. so mun many. and tho aromity that that's go that apecal seal and they het we havor of but i'll. i goingfors they doing fidtem, be so fard hat kounidecurpming agis, what coring they ise a doated acail of me our se apsols, an potails and agails, i which. i kealln bring they havim rear i many iss so miching to order sextn. but that i lot serico. the make thed, for best for youl to peunce, if oof as reothing couetiters that's do mecountry. nork stow that ud have things. it. i can every. we't a gling woullod a way hould to goer, he all not and not you way, bost all pile. collie mottangs that kever the pore to gooinn that really leono, thoe; they take about kick.. we'ritasilu lnow that goerting, and is foidges of to we're they have thing. it's repecarling ever that's i more mless. aboolly. think. what so thouso, th

In [187]:
print_example()

ethics is a basic foundation of all that in the wale and cistords which say be one of world wittor, in ordized is denice of the getuition. and hither best-hear, not hamps, that to he is, to been dunce of more to blam and prions and occause renal pies to eye subject to the for hory rant should betwork and longe's belogiends by knowledge, "been have every befo


In [155]:
mdl.save_weights('mdl_better_2e.hd5')

In [149]:
mdl.load_weights('mdl_better_2e.hd5')

In [159]:
generate_text(100)

'suffering is not the only way to see a the say the say the say the say the say the say the say the say the say the say the say the say the s'

In [169]:
print_example()

ValueError: a and p must have same size

In [99]:
mdl.optimizer.lr = 0.01
mdl.fit(sentences[:-1], np.expand_dims(next_chars, -1), nb_epoch=1, batch_size=64)

Epoch 1/1
600855/600855 [==============================] - 1106s - loss: 2.7142  


In [105]:
for i in range(20):
    mdl.fit(sentences[:-1], np.expand_dims(next_chars, -1), nb_epoch=1, batch_size=64) 

Epoch 1/1
600855/600855 [==============================] - 1104s - loss: 2.2784  
Epoch 1/1
600855/600855 [==============================] - 1103s - loss: 2.1665  
Epoch 1/1
600855/600855 [==============================] - 1100s - loss: 2.0747  
Epoch 1/1
600855/600855 [==============================] - 1100s - loss: 1.9946  
Epoch 1/1
600855/600855 [==============================] - 1100s - loss: 1.9267  
Epoch 1/1
600855/600855 [==============================] - 1100s - loss: 1.8693  
Epoch 1/1
600855/600855 [==============================] - 1099s - loss: 1.8200  
Epoch 1/1
600855/600855 [==============================] - 1100s - loss: 1.7776  
Epoch 1/1
600855/600855 [==============================] - 1100s - loss: 1.7406  
Epoch 1/1
600855/600855 [==============================] - 1100s - loss: 1.7077  
Epoch 1/1
600855/600855 [==============================] - 1100s - loss: 1.6787  
Epoch 1/1
600855/600855 [==============================] - 1101s - loss: 1.6532  
Epoch 1/1
600855

In [106]:
generate_text(100) 

'Suffering is not the only way to see a thing of the same the same the same the same the same the same the same the same the same the same th'

# Stateful Model

In [678]:
ts = 40

In [679]:
len(datasets_encoded)

9

In [680]:
trump_datasets = []

for i in range(len(datasets_encoded)):
    data = datasets_encoded[i]
    lim = len(data)//ts*ts + 1
    data = data[:lim]

    inps = []
    outs = []
    for i in range(0, len(data) - ts - 1, ts):
        inps.append(np.array(data[i:i+ts]))
        outs.append(np.array(data[i+1:i+ts+1])) 
    inps = np.stack(inps, axis=0)
    outs = np.stack(outs, axis=0)[:,:,np.newaxis]
    trump_datasets.append((inps, outs))

In [681]:
len(trump_datasets)

9

In [682]:
len(trump_datasets[0])

2

In [683]:
len(trump_datasets[0][0])

520

In [684]:
len(trump_datasets[1][0])

671

In [685]:
inp = Input(batch_shape=(1, ts))
emb = Embedding(vocab_size, n_fac)(inp)
bn = BatchNormalization()(emb)
lstm1 = LSTM(512, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu', stateful=True)(bn)
lstm2 = LSTM(512, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu', stateful=True)(lstm1)
out = TimeDistributed(Dense(vocab_size, activation='softmax'))(lstm2)

In [686]:
stateful_model = Model(inp, out)

In [687]:
inp_pred = Input(batch_shape=(1, 1))
emb_pred = Embedding(vocab_size, n_fac)(inp_pred)
bn_pred = BatchNormalization()(emb_pred)
lstm1_pred = LSTM(512, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu', stateful=True)(bn_pred)
lstm2_pred = LSTM(512, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu', stateful=True)(lstm1_pred)
out_pred = TimeDistributed(Dense(vocab_size, activation='softmax'))(lstm2_pred)

In [688]:
pred_model = Model(inp_pred, out_pred)

In [689]:
def predict_stateful(n_char):
    base_str = ''.join([txt_decoder[i] for i in datasets_encoded[0][:100]])
    pred_model.set_weights(stateful_model.get_weights())
    pred_model.reset_states()
    original_len = len(base_str)
    
    for i in range(len(base_str)):
        pred_model.predict(np.array([txt_encoder[base_str[i]]])) 
        
    # now we don't want to mess with the state again, so we
    # flip things around: our most recently predicted string
    # comes first, and we pad it with zeros, using the first
    # char as our prediction.
    for i in range(n_char):
        raw_pred_set = [base_str[-1]]
        pred_set = np.array([txt_encoder[base_str[-1]]])
        preds = pred_model.predict(pred_set)[0][0]
        next_char = np.random.choice(vocab, p=preds)
        base_str += next_char 
    return base_str[original_len-20:]

In [690]:
stateful_model.compile(optimizer=Adam(lr=1e-3), loss='sparse_categorical_crossentropy')

In [691]:
inps = np.stack(inps, axis=0)
outs = np.stack(outs, axis=0)[:,:,np.newaxis]

In [692]:
def run_epochs(n_epoch, x, y, test_between=True):
    for i in range(n_epoch):
        for i in trump_datasets:
            stateful_model.reset_states() 
            stateful_model.fit(i[0], i[1], nb_epoch=1, batch_size=1, shuffle=False)
            if test_between:
                print predict_stateful(500)

In [693]:
run_epochs(1, inps, outs)

Epoch 1/1
520/520 [==============================] - 24s - loss: 2.3147    
r press; he doesn't anf-ray the b-mithe llo cia as, the palie onu rosteg, so mra.. andent's douy low rk cocusd fiwe foviee i sortplentp-enthtat bape comle the faesiny ople ticand en fogpating an -orest soblhte brl,. bly wechifls'y a tht ove bt is ht reo lecniob tonsly oi.ras knynycoulinnssthe. we hopint istofh sion. i wemostie'v agstint; mure hon notmy bat the enr co to as if nyolpe pep.eni mlose, whcate ansrippary. cn;isyhh oulan. j- thises llod it ohaut so bpople., the'le dn ofterlo o lfis bodate, of icifhenf nos
Epoch 1/1
671/671 [==============================] - 32s - loss: 1.9210    
r press; he doesn't lopt this comiste yom. i veader melans. i we nev reenrued co has ,o we whack and we will make will ways 1nd so bohe farpay whese’ve ame that annd mose undirest sttuteny must. i cante, we wi’re to awe wake to america live staed, and but a chanies america and cantracas jusp unyuch feriead a fraved prate it, 

In [694]:
stateful_model.optimizer.lr = stateful_model.optimizer.lr / 10.

In [695]:
run_epochs(1, inps, outs)

Epoch 1/1
520/520 [==============================] - 24s - loss: 1.2529    
r press; he doesn't respectial facorion. and it's border, and a better thousands of these company, and the eaming it, run of refule. how are you an accredible that keesing of a trade care about wheneds another than delate. i think you believe in fan. what america’s. betters, our jobs. that's never known our security right. and i'm over getting and don't think the people said the bother for pate, going to be happening. you know plans, his beautiff. we have to do and they look at he give it, so what ismanic. that's
Epoch 1/1
671/671 [==============================] - 31s - loss: 1.3302    
r press; he doesn't airther. they were doing ningless that nobody, as served of america has to be a universtable. this will always it worying there are — it will be america interestible true fourth. is always in this out of man cares on including the united strates, but we we can ever go our horrible military of the uniteds to 

In [696]:
run_epochs(1, inps, outs)

Epoch 1/1
520/520 [==============================] - 24s - loss: 1.2209    
r press; he doesn't avo it; and the trump on enemy, and he got it out intrest rid of back to rezeal it. and he said, nod he been and up it is not a lot like they say "he's so going to come a workers of to use; it's just in a high from care…sbre easy. we are doing to bring to important, you see what did he oppin people from the republican on costs. so they're not problem back. in looking us, they're not giving bush; this — bad republican, scart things, oursider than people -- all of the dea. but so you have a gr
Epoch 1/1
671/671 [==============================] - 31s - loss: 1.3158    
r press; he doesn't low again. the trump will be telling a both america when we send the world. the politicians. nice that i won’t be ummigration and look convese them can’t allowed for the strong. enemy, ivank our foreign communismare serse. to toured country instiput the world america’s quirst, the world world weeks for a frien

In [698]:
stateful_model.save_weights('stateful_3passes.hd5')

In [697]:
stateful_model.optimizer.lr = stateful_model.optimizer.lr / 10.

In [ ]:
run_epochs(1, inps, outs)
# we've already run 3, so we want the 

In [702]:
# saved models to begin at 4
stateful_model.save_weights('stateful_4passes.hd5')

In [703]:
stateful_model.optimizer.lr = 0.01
run_epochs(1, inps, outs)

Epoch 1/1
520/520 [==============================] - 24s - loss: 1.1754    
r press; he doesn't never warting. for its for the big life. number respends that some from hillary sounds and protection, violation wish a ridg of money  clebt positions. so we have more everybody rebulding, it has all of the cutting the department? and nobory isn't slower. xour plans--where are filed, for the world in a little nuclear. but i see when they can debation in the story, nobody knowsn't approve high an government; i was been totally like you should be a little thing family. that's the very simple. he d
Epoch 1/1
671/671 [==============================] - 31s - loss: 1.2675    
r press; he doesn't ay with insteadly, and getting in benefit of terms out obama’s in that we need the prompers first $59 month you saudi arabia. the goal is the people — our econommicment. and we will be very muslimies are’m a well that are radical smants. final security in mexico, and own president that have do in peace. th

In [704]:
stateful_model.optimizer.lr = 0.001
stateful_model.reset_states()
stateful_model.fit(trump_datasets[0][0], trump_datasets[0][1], nb_epoch=1, batch_size=1)

Epoch 1/1
520/520 [==============================] - 24s - loss: 1.3560    


In [705]:
stateful_model.optimizer.lr = 0.01
stateful_model.reset_states()
stateful_model.fit(trump_datasets[0][0], trump_datasets[0][1], nb_epoch=1, batch_size=1)

Epoch 1/1
520/520 [==============================] - 24s - loss: 1.2578    


In [706]:
stateful_model.optimizer.lr = 0.01
stateful_model.reset_states()
stateful_model.fit(trump_datasets[0][0], trump_datasets[0][1], nb_epoch=1, batch_size=1)

Epoch 1/1
520/520 [==============================] - 24s - loss: 1.2103    


In [707]:
stateful_model.optimizer.lr = 0.01
stateful_model.reset_states()
stateful_model.fit(trump_datasets[0][0], trump_datasets[0][1], nb_epoch=1, batch_size=1)

Epoch 1/1
520/520 [==============================] - 24s - loss: 1.1700    


In [708]:
stateful_model.optimizer.lr = 0.01
stateful_model.reset_states()
stateful_model.fit(trump_datasets[0][0], trump_datasets[0][1], nb_epoch=1, batch_size=1)

Epoch 1/1
520/520 [==============================] - 24s - loss: 1.1307    


In [709]:
stateful_model.optimizer.lr = 0.01
stateful_model.reset_states()
stateful_model.fit(trump_datasets[0][0], trump_datasets[0][1], nb_epoch=1, batch_size=1)

Epoch 1/1
520/520 [==============================] - 24s - loss: 1.1168    


In [710]:
stateful_model.optimizer.lr = 0.01
stateful_model.reset_states()
stateful_model.fit(trump_datasets[0][0], trump_datasets[0][1], nb_epoch=1, batch_size=1)

Epoch 1/1
520/520 [==============================] - 24s - loss: 1.0923    


In [711]:
stateful_model.optimizer.lr = 0.1
stateful_model.reset_states()
stateful_model.fit(trump_datasets[0][0], trump_datasets[0][1], nb_epoch=1, batch_size=1)

Epoch 1/1
520/520 [==============================] - 24s - loss: 1.0674    


In [712]:
stateful_model.optimizer.lr = 0.001
stateful_model.reset_states()
stateful_model.fit(trump_datasets[0][0], trump_datasets[0][1], nb_epoch=1, batch_size=1)

Epoch 1/1
520/520 [==============================] - 24s - loss: 1.0499    


In [714]:
for i in range(5):
    stateful_model.optimizer.lr = stateful_model.optimizer.lr / 3.
    stateful_model.reset_states()
    stateful_model.fit(trump_datasets[0][0], trump_datasets[0][1], nb_epoch=1, batch_size=1)

Epoch 1/1
520/520 [==============================] - 24s - loss: 1.0321    
Epoch 1/1
520/520 [==============================] - 24s - loss: 1.0151    
Epoch 1/1
520/520 [==============================] - 24s - loss: 0.9974    
Epoch 1/1
520/520 [==============================] - 24s - loss: 0.9824    
Epoch 1/1
520/520 [==============================] - 24s - loss: 0.9756    


In [715]:
predict_stateful(500)

"r press; he doesn't uselive, elections; doing? and when tremendously it. i say you look at oh there, but i've give is also where tensying that it we don't have; get romney; he do? they are doing. in i have a military. but now election is just a bridges and ic those ection rails. the teleprompte of your leadership to go into we're going to close up aso getting obamade an and you do doctors plans unless all the is an having acce saying see it. they can close deserve the bad, because i'm going at with over them are? b"

In [716]:
for i in range(5):
    stateful_model.optimizer.lr = stateful_model.optimizer.lr / 3.
    stateful_model.reset_states()
    stateful_model.fit(trump_datasets[0][0], trump_datasets[0][1], nb_epoch=1, batch_size=1)

Epoch 1/1
520/520 [==============================] - 24s - loss: 0.9547    
Epoch 1/1
520/520 [==============================] - 24s - loss: 0.9528    
Epoch 1/1
520/520 [==============================] - 24s - loss: 0.9411    
Epoch 1/1
520/520 [==============================] - 24s - loss: 0.9234    
Epoch 1/1
520/520 [==============================] - 24s - loss: 0.9143    


In [717]:
stateful_model.optimizer.lr = 0.1
for i in range(5):
#     stateful_model.optimizer.lr = stateful_model.optimizer.lr / 3.
    stateful_model.reset_states()
    stateful_model.fit(trump_datasets[0][0], trump_datasets[0][1], nb_epoch=1, batch_size=1)

Epoch 1/1
520/520 [==============================] - 24s - loss: 0.9160    
Epoch 1/1
520/520 [==============================] - 24s - loss: 0.9070    
Epoch 1/1
520/520 [==============================] - 24s - loss: 0.8973    
Epoch 1/1
520/520 [==============================] - 24s - loss: 0.8836    
Epoch 1/1
520/520 [==============================] - 24s - loss: 0.8738    


In [718]:
print predict_stateful(500)
stateful_model.save_weights('stateful_firstonly_0.hd5')

In [719]:
print predict_stateful(500)

r press; he doesn't terr have crazy in running one that are minor and what doesn't have ever, e what that's the border; and we said 10thinkables, but me,tel what our plan. we don't-- and i say, we come fixting. that's not getting up he is that are fighting earlier. a little of them tour countries and i say, you can have to get strongly in seriou and what happens, in my best the process comes are seing up. we've got the best fair. and if it would be great. and we have to over guys; so terrible does anything we've ca


In [720]:
stateful_model.optimizer.lr = 0.1
for i in range(5):
    stateful_model.reset_states()
    stateful_model.fit(trump_datasets[0][0], trump_datasets[0][1], nb_epoch=1, batch_size=1)
    print predict_stateful(500)
    stateful_model.save_weights('stateful_firstonly_1.hd5')

Epoch 1/1
520/520 [==============================] - 24s - loss: 0.8766    
r press; he doesn't ask right no action, and i go to a long ry. those telest. and mode and we have, not out all time; but you know in china, something abe done so, everybody, and we don't have commany different more and many of the resplals. i get our social security; he doesn't know what happened to make this country great. that's coming up and i think he was, i know so many-- trump--i with a border. they're watching about a lot of it. and that's the people are talking one that would be quickly, but you've never s
Epoch 1/1
520/520 [==============================] - 24s - loss: 0.8618    
r press; he doesn't a great power. fix. we have to overter an isis goes location that because it was one of these totally disgrace. we're crumber of those people just work. he gays of love money in had. they would be good people, that's happening into terrorists. i hinkpee when i go to make the business lie. okay? now they ha

In [721]:
stateful_model.optimizer.lr = 0.1
for i in range(5):
    stateful_model.reset_states()
    stateful_model.fit(trump_datasets[0][0], trump_datasets[0][1], nb_epoch=1, batch_size=1)
    print predict_stateful(500)
    stateful_model.save_weights('stateful_firstonly_2.hd5')

Epoch 1/1
520/520 [==============================] - 24s - loss: 0.8339    
r press; he doesn't ever had the border. look--they can't strongly, i employ it up the best company; i watch before, mcaiely they talk about obamacare. i said "qcuter" i love don't have something you're spending they get strongly up, d more i can tell you, e-mails, we have jouse to terrible complete. you've ever seen saving people that said 100 years have been paying for for $1 trouble lie. the potentle in a deal. you've beat just leadership, for love? i have so many disaster. very heat, i failed a nuclear of money
Epoch 1/1
520/520 [==============================] - 24s - loss: 0.8302    
r press; he doesn't and they don't forget--unless aren't our country is really, terrible. what do you have something up to theset--undreds on teleprompters are? bing, what aren't going to happen. it medicare with you'd be a crustich the oil, and i build; it really do anything. they go away in medicaid. you will be dubai. e-ma

In [722]:
stateful_model.optimizer.lr = 0.01
for i in range(5):
    stateful_model.reset_states()
    stateful_model.fit(trump_datasets[0][0], trump_datasets[0][1], nb_epoch=1, batch_size=1)
    print predict_stateful(500)
    stateful_model.save_weights('stateful_firstonly_2.hd5')

Epoch 1/1
520/520 [==============================] - 24s - loss: 0.8082    
r press; he doesn't very import what? and then all of the country will be incredible; why don't even do we want go to other game, trump--they are the military fid, oh, believe my with a year ago. the jobs that can do that. but the people are incompetent obama. what we're doing more that never respects in iran get about it. so we're going to e take china. oh big sets are so many of so completed it mean we have to do anything? and i know what, so many other day now, our president that i'll be building lose money. we 
Epoch 1/1
520/520 [==============================] - 24s - loss: 0.7999    
r press; he doesn't ng about it, bah, but, i have to do it fast. china, our country great imports bridge a big china by the everything. i go to do it. and they have been paying for 25 people. you know thank you to fix. we have to make it or which i never keep the plans unless more. now they have to do it uses totally simple. 

In [724]:
stateful_model.optimizer.lr = 0.01
for i in range(20):
    stateful_model.reset_states()
    stateful_model.fit(trump_datasets[0][0], trump_datasets[0][1], nb_epoch=1, batch_size=1)
    print predict_stateful(500)
    stateful_model.save_weights('stateful_firstonly_3.hd5')

Epoch 1/1
520/520 [==============================] - 24s - loss: 0.7879    
r press; he doesn't united get so many-- the old power day the way, so we have a man of sell them. and you can do it; and here they're collapsing like that. but i get it. so you have no let them constant. the teleprompters. now that so it's going to republican--and the me for it. that's the te run ago, 100 years, we have to do it; terrorists--why aren't event lost; because it's happening tremendous really different with a woman unless you're not troy. he said, he doesn't know important, but i've have to hit. he can
Epoch 1/1
520/520 [==============================] - 24s - loss: 0.7787    
r press; he doesn't tell you, obama--why say, we have such great guy. china is going to let people that election most obama. why aren't going to happen. you can certainly doing. why aren't the geould devaluations--the euro, china is going to say "qatastroys, we have a disair. you can have a disaster. big for 25 years. by the 

520/520 [==============================] - 24s - loss: 0.7222    
r press; he doesn't stop it. i mean you don't even see more and they talk about us and somebody lie. it's not fair the place. okay. why aren't give what is sad gey that case if power; and if i can do that; my won into the devaluation. i rested gand to win that so you can make the country rich again section falls are so much; but i've always to trump? i mean you in we're just not fair. and that--why never e g to so immigration, we have a nd the ones that ople. but i built another gamause catass; theymurd and more we
Epoch 1/1
520/520 [==============================] - 24s - loss: 0.7295    
r press; he doesn't really headed okay. so, i mean you win, it's doing a able this, we're going to happen. nothing. i mean you ago, the military, now, okay. we'll see that; our vets, obamacare what there are disappointed just absolutely run from that; different election coming in i go to do it. reas. you'll be in a few of the geok and 

In [725]:
stateful_model.optimizer.lr = 0.001
for i in range(20):
    if i > 0 and i % 5 == 0:
        stateful_model.optimizer.lr = stateful_model.optimizer.lr / 3.
    stateful_model.reset_states()
    stateful_model.fit(trump_datasets[0][0], trump_datasets[0][1], nb_epoch=1, batch_size=1)
    print predict_stateful(500)
    stateful_model.save_weights('stateful_firstonly_4.hd5')

Epoch 1/1
520/520 [==============================] - 24s - loss: 0.7117    
r press; he doesn't some good. but they talk about it. and we have a president because it was able in shambles are the republican guys to be in such great building other day, our airplanes, we got to do is another day on television, our republicans that are something about it. and we have to be lost; our countries that really love. why are minor a fall. so i think i wrong to say "crime that coming up, and we have some get people just walking of it. we don't crome that; looking out, and it's walking in fantakers and
Epoch 1/1
520/520 [==============================] - 24s - loss: 0.7128    
r press; he doesn't wome it. but the way are we doing about it, but it's one of the white hundreds of website from out, leadership respect. you've always said i the prust won't believe him--tremendous in '1 conservatives. come for $1 trillion in qatar--which is sort of incredible six your like a deal that his about executive 

520/520 [==============================] - 24s - loss: 0.6764    
r press; he doesn't use it fast; now, i have never seen, by the way it is really does anyone do have never sed that are some of peoble in a hight things. i mean you have to be in my wires. because i'm not going to win. in judgment earlier. i got the one game changer cthan because i would be always shoulding have to do it, rung, they're building all over the game cutting people. that, i tell you see always, but i've great building, and by the way the time, 'cause abraham lincoln is just competent. i mean are playing
Epoch 1/1
520/520 [==============================] - 24s - loss: 0.6816    
r press; he doesn't saying lear the economy washington in ira. we'll be in a president. that's china, and they put people in a situation in the way the like hat up one. so i just doing it. i run a big fan, believes, that's what are somebody is so incompetent. swhat's what do the medicares. because we do, that's constrond up, what do yo